In [1]:
# bibliotecas
import math, random
import matplotlib
import decimal
import pandas as pd

In [2]:
# lendo os arquivos de entrada
initial_weights = open('initial_weights.txt','r+').readlines()

def read_dataset(path_file):
    '''função responsável em ler o arquivo de dataset'''
    dataset = list()
    file = open(path_file, 'r+')
    for lines in file:
        values_line = list()
        dataset.append([lines.rstrip()])
    return dataset
    
def read_def_network(path_file):
    '''função responsável em ler o arquivo de definição da rede neural'''
    file = open(path_file, "r+")
    
    network = list()
    for line in file:
        network.append(line.rstrip())
    return network

dataset = read_dataset("dataset.txt")
print('dataset:\n', dataset)

def_network = read_def_network('network.txt')
print('def_network:\n', def_network)

dataset:
 [['0.32000, 0.68000; 0.75000, 0.98000'], ['0.83000, 0.02000; 0.75000, 0.28000']]
def_network:
 ['0.250', '2', '4', '3', '2']


#### Rede neural composta por um número ajustável de neurônios e camadas e treinada via backpropagation

In [3]:
def create_network(def_network, initial_weights):
    '''função responsável em criar a rede neural a partir dos arquivos de definição'''
    '''def_network definição da estrutura da rede'''
    '''initial_weights pesos inicias da rede'''
    network = list()
    
    # trabalhando sobre o arquivo que define a rede
    regulation = def_network[0]
    def_network.pop(0) # remove a regulação já armazenada
    
    input_length = def_network[0]
    def_network.pop(0)
    
    output_length = def_network[len(def_network)-1]
    def_network.pop(len(def_network)-1)
    
    # imprimindo informações capturadas da definição da rede
    print('[INFO] Regulação: ', regulation)
    print('[INFO] Quantidade de entradas: ', input_length)
    print('[INFO] Quantidade de saídas: ', output_length)
    # imprimindo informações das camadas ocultas
    print('[INFO] Camadas ocultas: ', def_network, '\n')
    
    # trabalhando sobre o arquivo de quefine os pesos
    for weights in initial_weights:
        layers = weights.rstrip().split('; ')
        network_layer = list()
        for neuron in layers:
            if ', ' in neuron:
                network_layer.append({'weights': neuron.rstrip().split(', ')})
            else:
                network_layer.append(neuron)
        network.append(network_layer)
    print('[INFO] Rede neural criada: ', network)
    return network

# construindo a rede neural
network = create_network(def_network, initial_weights)  # TODO: mudar para o arquivo def a rede


[INFO] Regulação:  0.250
[INFO] Quantidade de entradas:  2
[INFO] Quantidade de saídas:  2
[INFO] Camadas ocultas:  ['4', '3'] 

[INFO] Rede neural criada:  [[{'weights': ['0.42000', '0.15000', '0.40000']}, {'weights': ['0.72000', '0.10000', '0.54000']}, {'weights': ['0.01000', '0.19000', '0.42000']}, {'weights': ['0.30000', '0.35000', '0.68000']}], [{'weights': ['0.21000', '0.67000', '0.14000', '0.96000', '0.87000']}, {'weights': ['0.87000', '0.42000', '0.20000', '0.32000', '0.89000']}, {'weights': ['0.03000', '0.56000', '0.80000', '0.69000', '0.09000']}], [{'weights': ['0.04000', '0.87000', '0.42000', '0.53000']}, {'weights': ['0.17000', '0.10000', '0.95000', '0.69000']}]]


#### Funcionalidade que permita, via linha de comando, informar a sua implementação a estrutura de uma rede de teste (i.e., estrutura de camadas/neurônios, pesos iniciais, e fator de regularização), e um conjunto de treinamento, e que retorne o gradiente calculado para cada peso

#### Funcionalidade que permita, via linha de comando, efetuar a verificação numérica do gradiente, a fim de checar a corretude da implementação de cada grupo

#### Um mecanismo para normalização das features/dados de treinamentos

#### Mecanismo para uso de regularização